# Ship selection
Some ships may have strange shapes or stabilizing fins so that they have roll decay results that are very much different.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
#from jupyterthemes import jtplot
#jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay import database
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import run
from rolldecay.froude_scaling import froude_scale
from mdldb.tables import Run, RollDecayExclude
from rolldecay.paper_writing import save_fig

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.90)
db = database.get_db()

In [ ]:
6058 in df_rolldecay.index

In [ ]:
df_rolldecay.describe()

In [ ]:
ids = ''
unique_ids = df_rolldecay['loading_condition_id'].unique()
for id in unique_ids[0:-1]:
    ids+='%i,'%id
ids+='%i' % unique_ids[-1]

sql = """
select * 
from loading_conditions
INNER JOIN models
ON loading_conditions.model_number == models.model_number
    INNER JOIN ships
    ON models.ship_name == ships.name
        INNER JOIN association
        ON ships.name==association.ship_name 
            INNER JOIN projects
            ON association.project_number = projects.project_number
        
where loading_conditions.id in (%s)
""" % ids
loading_conditions = pd.read_sql(sql, con=db.engine, index_col='id', )
loading_conditions = loading_conditions.loc[:, ~loading_conditions.columns.duplicated()]

In [ ]:
mask = loading_conditions['lpp']==0
loading_conditions.loc[mask]

In [ ]:
T_f=loading_conditions['TF']
T_a=loading_conditions['TA']
L_pp=loading_conditions['lpp']
loading_conditions['trim']=np.rad2deg(np.arctan((T_a-T_f)/L_pp))

limit=0.4
mask = loading_conditions['trim'].abs() > limit
loading_conditions.loc[mask][['TF','TA','lpp','trim']]

In [ ]:
loading_conditions.describe()

In [ ]:
mask = loading_conditions['trim'].abs() < 0.4
loading_conditions=loading_conditions.loc[mask].copy()
loading_conditions.describe()

In [ ]:
description = pd.read_sql_table('description', con=db.engine, index_col='id')
description.loc['ship_speed','unit']='m/s'
description.loc['VDES','unit']='m/s'

In [ ]:
skip = [
    'model_number',
    'loading_condition_id',
    'B_1A',
    'B_2A',
    'B_3A',
    'C_1A',
    'C_3A',
    'C_5A',
    'B_1A',
    'B_1',
    'B_2',
    'B_3',
    'C_1',
    'C_3',
    'C_5',
    'A_44',
    'omega0_fft',
    'omega0',
    'score',
    'id',
    'project_number',
    'series_number',
    'run_number',
    'test_number',
    'scale_factor',
    'g',
    'rho',
    'B_1_hat',
    'B_2_hat',
    'CB',
    'trim',
]

In [ ]:
loading_conditions

In [ ]:
volume = loading_conditions['Volume']
lpp = loading_conditions['lpp']
TA = loading_conditions['TA']
TF = loading_conditions['TF']
beam = loading_conditions['beam']
T = (TA+TF)/2
CB = volume / (lpp*beam*T)
loading_conditions['CB']=CB
mask = loading_conditions['CB'].abs()!=np.inf
loading_conditions=loading_conditions.loc[mask].copy()

In [ ]:
df_model = froude_scale(data=loading_conditions, description=description, skip=skip)
df_model['scale_factor']=df_model['lpp']
mask = df_model['scale_factor']!=0
df_model=df_model.loc[mask]
loading_conditions_1 = froude_scale(data=df_model, description=description, skip=skip)

In [ ]:
loading_conditions_1.describe()

In [ ]:
len(loading_conditions)

In [ ]:
len(loading_conditions_1)

In [ ]:
loading_conditions_1['gm'].hist(bins=50)

In [ ]:
mask = loading_conditions_1['gm'] > loading_conditions_1['gm'].quantile(0.999)

In [ ]:
loading_conditions_1.loc[mask].iloc[0]

In [ ]:
good = loading_conditions_1['gm'] < loading_conditions_1['gm'].quantile(0.999)
loading_conditions_1=loading_conditions_1.loc[good].copy()
loading_conditions_1.describe()

In [ ]:
loading_conditions_1['KXX'].hist(bins=50)

In [ ]:
loading_conditions_1['beam'].hist(bins=50)

In [ ]:
loading_conditions_1['A0'].hist(bins=50)

In [ ]:
loading_conditions_1['A0'].quantile(0.001)

In [ ]:
mask = loading_conditions_1['A0'] < loading_conditions_1['A0'].quantile(0.001)

In [ ]:
loading_conditions_1.loc[mask].iloc[0]

In [ ]:
good = loading_conditions_1.index != loading_conditions_1.loc[mask].iloc[0].name
loading_conditions_1=loading_conditions_1.loc[good].copy()
loading_conditions_1.describe()

In [ ]:
loading_conditions_1['CB'].hist(bins=50)

In [ ]:
mask = loading_conditions_1['CB'] > 0.5
loading_conditions_1 = loading_conditions_1.loc[mask].copy()
loading_conditions_1.describe()

In [ ]:
loading_conditions_1['TA'].hist(bins=50)

In [ ]:
ikeda_parameters = ['beam','DRAFT','BKL','BKB','kg','CB','A0']
loading_conditions_1['DRAFT']=loading_conditions_1['TA']
fig,axes=plt.subplots(nrows=len(ikeda_parameters))
for parameter,ax in zip(ikeda_parameters,axes):
    loading_conditions_1[parameter].hist(bins=50, ax=ax)
    ax.set_xlabel(parameter)

axes[0].set_title('Histogram of ships');
save_fig(fig=fig, name='ship_parameters', full_page=True)


In [ ]:
loading_conditions_1['TA'].min()

In [ ]:
removes = set(loading_conditions.index) - set(loading_conditions_1.index)
removes

In [ ]:
#df_ikeda['g']=9.81
#df_ikeda['rho']=1000
#df_ikeda['Disp']=df_ikeda['Volume']/(df_ikeda['scale_factor']**3)
#df_ikeda['B_1_hat'] = run(function=lambdas.B_1_hat_lambda, inputs=df_ikeda)
#df_ikeda['B_2_hat'] = run(function=lambdas.B_2_hat_lambda, inputs=df_ikeda)

In [ ]:
df_rolldecay_exclude = pd.DataFrame()
for remove in removes:
    mask = df_rolldecay['loading_condition_id']==remove
    df_rolldecay_exclude=df_rolldecay_exclude.append(df_rolldecay.loc[mask])

In [ ]:
df_rolldecay.describe()

In [ ]:
df_rolldecay_exclude.describe()

In [ ]:
6058 in df_rolldecay.index

In [ ]:
6058 in df_rolldecay_exclude.index

## Save outliers:

In [ ]:
#roll_decay_exclude=RollDecayExclude(run_id=1)
#db.session.merge(roll_decay_exclude)
#db.session.commit()
#RollDecayExclude.__table__.drop(db.engine)
#RollDecayExclude.metadata.create_all(db.engine)

for run_id, _ in df_rolldecay_exclude.iterrows():
    roll_decay_exclude = RollDecayExclude(run_id=run_id)
    db.session.merge(roll_decay_exclude)
    
db.session.commit()

In [ ]:
df_rolldecay['ship_type_id'].unique()

In [ ]:
df_shiptypes = pd.read_sql_table(table_name='shiptype', con=db.engine, index_col='id')
df_shiptypes = df_shiptypes.append(pd.Series(data={'description':'unknown'}, name=0))

In [ ]:
df_shiptypes

In [ ]:
df_rolldecay['ship_type_id'].fillna(0, inplace=True)

In [ ]:
shiptype_count = df_rolldecay.groupby(by='ship_type_id')[['model_number']].count()
shiptype_count.rename(columns={'model_number':'Number of tests'}, inplace=True)
shiptype_count['Ship type'] = df_shiptypes.loc[shiptype_count.index]

In [ ]:
shiptype_count[['Number of tests','Ship type']]

In [ ]:
latex = shiptype_count[['Ship type','Number of tests']].to_latex(index=False)

In [ ]:
print(latex)

In [ ]:
fig,ax=plt.subplots()
shiptype_count.plot(kind='bar', x='Ship type', y='Number of tests', ax=ax)
ax.set_title('Number of tests for each ship type')
save_fig(fig=fig, name='ship_types')